In [ ]:
sc.install_pypi_package("scipy==1.1.0")
sc.install_pypi_package("Cython==0.29.33")
sc.install_pypi_package("scikit-learn==0.24.2")
sc.install_pypi_package("pandas==0.23.2")
sc.install_pypi_package("s3fs==0.4.2")
sc.install_pypi_package("boto3==1.26.59")
sc.install_pypi_package("joblib==1.2.0")

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from itertools import groupby
from operator import itemgetter

import pandas as pd
import boto3
import os
import pickle
import tempfile
import joblib
import io

MongoUrl = "mongodb+srv://xxxxxxxxxxxxxxxxxxxxxx/"

In [ ]:
spark = SparkSession \
    .builder \
    .appName("Frequency") \
    .config("spark.cores.max", 6) \
    .config("spark.executor.cores", 6) \
    .config("spark.executor.memory", "36g") \
    .getOrCreate()

In [ ]:
df = spark.read.format("mongodb") \
    .option("spark.mongodb.read.connection.uri", MongoUrl) \
    .option("spark.mongodb.write.connection.uri", MongoUrl) \
    .option("database", "coreEngine") \
    .option("collection", "PriceFeatures") \
    .option("aggregation.pipeline", "{ $match: { updateDate: { $gte: '20170101' } } }") \
    .load() \
    .drop("_id")

In [ ]:
historicalDay = df.select("updateDate").distinct().orderBy("updateDate")

In [ ]:
historicalDayList = [i[0] for i in historicalDay.toPandas().values if i[0] > '20221205']

In [ ]:
hf = df.toPandas()

In [ ]:
s3 = boto3.client(
    "s3",
    region_name="ap-northeast-2",
    aws_access_key_id="xxxxxxxxxxxxxxxxxxxxxx",
    aws_secret_access_key="xxxxxxxxxxxxxxxxxxxxxx")

In [ ]:
hf.updateDate = hf.updateDate.astype('int32')

In [ ]:
for idx, dt in enumerate(historicalDayList):
    boundary = int(dt)
    if boundary >= 20170102:
        hf.updateDate = hf.updateDate.astype('int32')
        mask_train = hf.updateDate <= boundary
        df_train = hf.loc[mask_train,:].sort_values(by=['stockCode', 'updateDate'], ascending=True).set_index(["stockCode", "stockFullName", "updateDate"])

        df_train_event = df_train[["event"]]
        df_train_set = df_train.drop(labels="event", axis=1)
        model = make_pipeline(MinMaxScaler(), LogisticRegression(class_weight="balanced"))
        model.fit(df_train_set, df_train_event)

  //      f = io.BytesIO()
  //      joblib.dump(model, f)
 //       f.seek(0)
 //       s3.put_object(Bucket="penta-engine", Key=f"HighFrequencyModel/{dt}.pkl", Body=f)
